In [12]:
import sys
sys.path.append('/home/diego/Git/thesis-tabtrans')
project_path = '/home/diego/Git/thesis-tabtrans'
from utils import data
import os
import numpy as np
import torch
import torch.nn as nn
from utils import training, callback, evaluating, attention, data, plots, fast_model
from sklearn import datasets, model_selection
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import GridSearchCV

import skorch
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import openml
from sklearn import datasets, model_selection
from skorch.callbacks import Checkpoint, EarlyStopping, LoadInitState, EpochScoring, Checkpoint, TrainEndCheckpoint
import csv
import xgboost as xgb

In [7]:
task_id = 233090 #anneal


X_train, X_test, y_train, y_test, train_indices, val_indices, _, n_labels, n_numerical, n_categories = data.import_data(task_id, 100)


INFO:openml.datasets.dataset:pickle write anneal


In [8]:
# Create training set

# Create validation set
X_val = X_train[val_indices]
y_val = y_train[val_indices]


#Update to have the train an validation sets separated
X_train = X_train[train_indices] 
y_train = y_train[train_indices]



In [9]:
clf_xgb = xgb.XGBClassifier(objective='multi:softmax', num_class=n_labels, seed = 11, device= 'cuda')
clf_xgb.fit(X_train, 
            y_train,
            verbose = True, #displays training progress information on the console.
            early_stopping_rounds = 10,
            eval_metric = "mlogloss", #multiclass log loss
            eval_set = [(X_val, y_val)])

[0]	validation_0-mlogloss:0.98522
[1]	validation_0-mlogloss:0.68290
[2]	validation_0-mlogloss:0.49285
[3]	validation_0-mlogloss:0.36504
[4]	validation_0-mlogloss:0.27559
[5]	validation_0-mlogloss:0.21192
[6]	validation_0-mlogloss:0.16615
[7]	validation_0-mlogloss:0.12986
[8]	validation_0-mlogloss:0.10519
[9]	validation_0-mlogloss:0.08711
[10]	validation_0-mlogloss:0.07414
[11]	validation_0-mlogloss:0.06362
[12]	validation_0-mlogloss:0.05695
[13]	validation_0-mlogloss:0.05042
[14]	validation_0-mlogloss:0.04791
[15]	validation_0-mlogloss:0.04419
[16]	validation_0-mlogloss:0.04287
[17]	validation_0-mlogloss:0.04108
[18]	validation_0-mlogloss:0.03848
[19]	validation_0-mlogloss:0.03731
[20]	validation_0-mlogloss:0.03573
[21]	validation_0-mlogloss:0.03487
[22]	validation_0-mlogloss:0.03348
[23]	validation_0-mlogloss:0.03317
[24]	validation_0-mlogloss:0.03327
[25]	validation_0-mlogloss:0.03338
[26]	validation_0-mlogloss:0.03367
[27]	validation_0-mlogloss:0.03373
[28]	validation_0-mlogloss:0.0

/home/diego/anaconda3/envs/tabtrans/lib/python3.11/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/home/diego/anaconda3/envs/tabtrans/lib/python3.11/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=6,
              num_parallel_tree=None, ...)

In [11]:
y_pred = clf_xgb.predict(X_test)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {balanced_accuracy:.4f}")

Balanced Accuracy: 0.8985


In [17]:
#Now I will improve parameters


task_id = 233090 #anneal
X_train, X_test, y_train, y_test, train_indices, val_indices, _, n_labels, n_numerical, n_categories = data.import_data(task_id, 100)



param_grid = {
    'max_depth': [3, 4, 5],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.01, 0.05],
    'gamma': [0, 0.25, 1.0],
    'subsample': [0.8, 1],
    'reg_lambda': [0, 1.0, 10.0],
    'colsample_bytree': [0.6, 0.8, 0.5]
}

clf_xgb = xgb.XGBClassifier(objective='multi:softmax', num_class=n_labels, seed = 11, device= 'cuda')

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=clf_xgb,
                          param_grid=param_grid,
                          scoring='balanced_accuracy',  # Adjust scoring metric as needed
                          cv=5,  # 5-fold cross-validation
                          n_jobs=-1)  # Use all cores

# Fit the grid search to the training set
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
print(best_params)

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {balanced_accuracy:.4f}")

INFO:openml.datasets.dataset:pickle write anneal


{'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 300, 'reg_lambda': 0, 'subsample': 0.8}
Balanced Accuracy: 0.9000
